In [ ]:
# | default_exp _code_generator.app_and_test_generator

In [ ]:
# | export

from typing import *
import time
import importlib.util
from tempfile import TemporaryDirectory
from pathlib import Path
import platform
import subprocess  # nosec: B404: Consider possible security implications associated with the subprocess module.

from yaspin import yaspin

from faststream_gen._components.logger import get_logger
from faststream_gen._code_generator.helper import (
    CustomAIChat,
    ValidateAndFixResponse,
    write_file_contents,
    read_file_contents,
    validate_python_code,
)
from faststream_gen._code_generator.prompts import APP_AND_TEST_GENERATION_PROMPT
from faststream_gen._code_generator.constants import (
    APPLICATION_SKELETON_FILE_NAME,
    APPLICATION_FILE_NAME,
    INTEGRATION_TEST_FILE_NAME,
)

In [ ]:
import pytest

from faststream_gen._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def _split_app_and_test_code(response: str) -> Tuple[str, str]:
    app_code, test_code = response.split("### application.py ###")[1].split("### test.py ###")
    return app_code, test_code


def _validate_response(response: str) -> List[str]:
    try:
        app_code, test_code = _split_app_and_test_code(response)
    except (IndexError, ValueError) as e:
        return [
            "Please add ### application.py ### and ### test.py ### in your response"
        ]
    with TemporaryDirectory() as d:
        write_file_contents(
            f"{d}/{APPLICATION_FILE_NAME}",
            app_code.replace("### application.py ###", ""),
        )

        test_file = f"{d}/{INTEGRATION_TEST_FILE_NAME}"
        write_file_contents(test_file, test_code)

        cmd = ["pytest", test_file, "--tb=short"]
        # nosemgrep: python.lang.security.audit.subprocess-shell-true.subprocess-shell-true
        p = subprocess.run(  # nosec: B602, B603 subprocess call - check for execution of untrusted input.
            cmd,
            stderr=subprocess.PIPE,
            stdout=subprocess.PIPE,
            shell=True if platform.system() == "Windows" else False,
        )
        if p.returncode != 0:
            return [str(p.stdout.decode("utf-8"))]

        return []

In [ ]:
fixture_response = """
### application.py ###

print('hi')

### test.py ###
def test_always_passes():
    assert True
"""

expected = []
actual = _validate_response(fixture_response)
print(actual)
assert actual == expected

[]


In [ ]:
fixture_response = """
### application.py ##

print('hi')

### test.py ###
def test_always_passes():
    assert True
"""
expected = ['Please add ### application.py ### and ### test.py ### in your response']
actual = _validate_response(fixture_response)
print(actual)
assert actual == expected

['Please add ### application.py ### and ### test.py ### in your response']


In [ ]:
fixture_response = """
### application.py ###

print('hi')

### test.py ###

def test_always_fails():
    assert False
"""
fixture_app_code = "print('hi')"

actual = _validate_response(fixture_response)
print(actual[0])
assert actual != []
print("OK")

============================= test session starts ==============================
platform linux -- Python 3.11.4, pytest-7.4.0, pluggy-1.3.0
rootdir: /tmp/tmpbhn0i5qi
plugins: anyio-3.7.1, asyncio-0.21.1
asyncio: mode=Mode.STRICT
collected 1 item

../../../tmp/tmpbhn0i5qi/test.py F                                       [100%]

=================================== FAILURES ===================================
______________________________ test_always_fails _______________________________
/tmp/tmpbhn0i5qi/test.py:4: in test_always_fails
    assert False
E   assert False
=========================== short test summary info ============================
FAILED ../../../tmp/tmpbhn0i5qi/test.py::test_always_fails - assert False
============================== 1 failed in 0.04s ===============================

OK


In [ ]:
# | export


def generate_app_and_test(
    description: str,
    model: str,
    code_gen_directory: str,
    total_usage: List[Dict[str, int]],
    relevant_prompt_examples: str,
) -> List[Dict[str, int]]:
    """Generate integration test for the FastStream app

    Args:
        description: Validated User application description
        code_gen_directory: The directory containing the generated files.
        relevant_prompt_examples: Relevant examples to add in the prompts.

    Returns:
        The generated integration test code for the application
    """
    with yaspin(
        text="Generating application and tests (usually takes around 30 to 40 seconds)...", color="cyan", spinner="clock"
    ) as sp:
        app_file_name = f"{code_gen_directory}/{APPLICATION_SKELETON_FILE_NAME}"
        app_skeleton = read_file_contents(app_file_name)

        prompt = (
            APP_AND_TEST_GENERATION_PROMPT.replace(
                "==== REPLACE WITH APP DESCRIPTION ====", description
            )
            .replace("==== RELEVANT EXAMPLES GOES HERE ====", relevant_prompt_examples)
            .replace("from .app import", "from application import")
        )
        test_generator = CustomAIChat(
            params={
                "temperature": 0.5,
            },
            model=model,
            user_prompt=prompt,
            semantic_search_query="How to test FastStream applications? Explain in detail.",  # todo: experiment without this query
        )
        test_validator = ValidateAndFixResponse(test_generator, _validate_response)
        validated_app_and_test_code, total_usage = test_validator.fix(
            f"{app_skeleton}",
            total_usage=total_usage,
        )
        
        app_code, test_code = _split_app_and_test_code(validated_app_and_test_code)
        
        app_output_file = f"{code_gen_directory}/{APPLICATION_FILE_NAME}"
        write_file_contents(app_output_file, app_code)

        test_output_file = f"{code_gen_directory}/{INTEGRATION_TEST_FILE_NAME}"
        write_file_contents(test_output_file, test_code)

        sp.text = ""
        sp.ok(f" ✔ The app and the tests are generated.")
        return total_usage

In [ ]:
# | notest

fixture_skeleton_code = """
from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Product(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    currency: str = Field(..., description="Currency of the price")
    price: float = Field(..., description="Price of the product")


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("change_currency")
@broker.subscriber("store_product")
async def on_store_product(product: Product, logger: Logger) -> Product:
    '''Processes a message from 'store_product' topic, changes currency to 'EUR' and divides price by 7.5 if currency is 'HRK'.

    Instructions:
    1. Consume a message from 'store_product' topic.
    2. Log the consumed message using logger.info.
    3. Check if the currency attribute is set to 'HRK'.
    4. If the currency is 'HRK', change the currency to 'EUR' and divide the price by 7.5.
    5. If the currency is not 'HRK', do not modify the original message.
    6. Publish the consumed message to 'change_currency' topic.


    '''
    raise NotImplementedError()
"""

fixture_description = """
Create a FastStream application using localhost broker for testing and use default port number. It should consume from 'store_product' topic an JSON encoded object with the following three attributes: product_name, currency and price. The format of the currency will be three letter string, e.g. 'EUR'. For each consumed message, check if the currency attribute is set to 'HRK'. If it is then change the currency to 'EUR' and divide the price by 7.5, if the currency is not set to 'HRK' don't change the original message. Finally, publish the consumed message to 'change_currency' topic.
"""

relevant_examples = '''

==== EXAMPLE APP DESCRIPTION ====

Develop a FastStream application using localhost broker.
It should consume messages from 'course_updates' topic where the message is a JSON encoded object including two attributes: course_name and new_content.
If new_content attribute is set, then construct a new message appending 'Updated: ' before the course_name attribute.
Finally, publish this message to the 'notify_updates' topic.

==== EXAMPLE APP SKELETON ====

from typing import Optional

from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class CourseUpdates(BaseModel):
    course_name: str = Field(..., examples=["Biology"], description="Course example")
    new_content: Optional[str] = Field(
        default=None, examples=["New content"], description="Content example"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("notify_updates")
@broker.subscriber("course_updates")
async def on_course_update(msg: CourseUpdates, logger: Logger) -> CourseUpdates:
    """
    Processes a message from the 'course_updates' topic, If new_content attribute is set, then constructs a new message appending 'Updated: ' before the course_name attribute.
    Finally, publishes the message to the 'notify_updates' topic.

    Instructions:
    1. Consume a message from 'course_updates' topic.
    2. Create a new message object (do not directly modify the original).
    3. Processes a message from the 'course_updates' topic, If new_content attribute is set, then constructs a new message appending 'Updated: ' before the course_name attribute.
    4. Publish the modified message to 'notify_updates' topic.

    """
    raise NotImplementedError()

==== YOUR RESPONSE ====

### application.py ###

from typing import Optional

from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class CourseUpdates(BaseModel):
    course_name: str = Field(..., examples=["Biology"], description="Course example")
    new_content: Optional[str] = Field(
        default=None, examples=["New content"], description="Content example"
    )


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("notify_updates")
@broker.subscriber("course_updates")
async def on_course_update(msg: CourseUpdates, logger: Logger) -> CourseUpdates:
    logger.info(msg)

    if msg.new_content:
        logger.info(f"Course has new content {msg.new_content=}")
        msg = CourseUpdates(
            course_name=("Updated: " + msg.course_name), new_content=msg.new_content
        )
    return msg
    
### test.py ###

import pytest

from faststream.kafka import TestKafkaBroker

from .app import CourseUpdates, broker, on_course_update


@broker.subscriber("notify_updates")
async def on_notify_update(msg: CourseUpdates):
    pass


@pytest.mark.asyncio
async def test_app_without_new_content():
    async with TestKafkaBroker(broker):
        await broker.publish(CourseUpdates(course_name="Biology"), "course_updates")
        on_course_update.mock.assert_called_with(
            dict(CourseUpdates(course_name="Biology"))
        )
        on_notify_update.mock.assert_called_with(
            dict(CourseUpdates(course_name="Biology"))
        )


@pytest.mark.asyncio
async def test_app_with_new_content():
    async with TestKafkaBroker(broker):
        await broker.publish(
            CourseUpdates(
                course_name="Biology", new_content="We have additional classes..."
            ),
            "course_updates",
        )
        on_course_update.mock.assert_called_with(
            dict(
                CourseUpdates(
                    course_name="Biology", new_content="We have additional classes..."
                )
            )
        )
        on_notify_update.mock.assert_called_with(
            dict(
                CourseUpdates(
                    course_name="Updated: Biology",
                    new_content="We have additional classes...",
                )
            )
        )

'''

with TemporaryDirectory() as d:
    output_path = f"{str(d)}/fastkafka-gen"
    test_file = f"{output_path}/{APPLICATION_SKELETON_FILE_NAME}"    
    write_file_contents(test_file, fixture_skeleton_code)
    
    usage = generate_app_and_test(fixture_description, output_path, [], relevant_examples)
    
    assert Path(output_path).exists()
    
    actual = [file for file in Path(output_path).iterdir()]
    print(actual)
    assert len(actual) == 3
    
    output_file = f"{output_path}/{APPLICATION_FILE_NAME}"
    contents = read_file_contents(output_file)
    print(contents)
    
    output_file = f"{output_path}/{INTEGRATION_TEST_FILE_NAME}"
    contents = read_file_contents(output_file)
    print(contents)

assert int(usage[0]["total_tokens"]) > 0
print(usage)

⠋ Generating application and tests...[INFO] faiss.loader: Loading faiss with AVX2 support.
[INFO] faiss.loader: Successfully loaded faiss with AVX2 support.
⠙ Generating application and tests... 

/home/harish/.local/lib/python3.11/site-packages/yaspin/core.py:119: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✔ The app and the tests are generated and saved at: /tmp/tmp5d_c281v/fastkafka-gen/application.py and /tmp/tmp5d_c281v/fastkafka-gen/test.py 
[PosixPath('/tmp/tmp5d_c281v/fastkafka-gen/application_skeleton.py'), PosixPath('/tmp/tmp5d_c281v/fastkafka-gen/application.py'), PosixPath('/tmp/tmp5d_c281v/fastkafka-gen/test.py')]


from pydantic import BaseModel, Field

from faststream import FastStream, Logger
from faststream.kafka import KafkaBroker


class Product(BaseModel):
    product_name: str = Field(..., description="Name of the product")
    currency: str = Field(..., description="Currency of the price")
    price: float = Field(..., description="Price of the product")


broker = KafkaBroker("localhost:9092")
app = FastStream(broker)


@broker.publisher("change_currency")
@broker.subscriber("store_product")
async def on_store_product(product: Product, logger: Logger) -> Product:
    logger.info(product)

    if product.currency == "HRK":
        logger.info(f"Currency is HRK. Chang